In [11]:
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models 
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = mnist.load_data()
images, labels = (x_train[0:1000].reshape(1000, 28*28) / 255, y_train[0:1000])

one_hot_labels = np.zeros((len(labels), 10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test), 28*28) / 255
test_labels = np.zeros((len(y_test), 10))

for i,l in enumerate(y_test):
    test_labels[i][l] = 1

def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1-(output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

np.random.seed(1)
alpha, iterations, hidden_size = (0.02, 300, 40)
pixels_per_image, num_labels = (784, 10)
batch_size = 100

weights_0_1 = 0.02 * np.random.random((pixels_per_image, hidden_size)) - 0.01
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

for j in range(iterations):
    train_acc = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i+1) * batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_1 = tanh(np.dot(layer_0, weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1, weights_1_2))

        for k in range(batch_size):
            train_acc += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))

        layer_2_delta = (labels[batch_start:batch_end] - layer_2) / batch_size
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask

        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    test_acc = 0

    for i in range(len(test_images)):
        layer_0 = test_images[i:i+1]
        layer_1 = tanh(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        test_acc += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

    if (j % 10 == 0):
        sys.stdout.write("\n" + "I:" + str(j) + "Train-ACC:" + str(train_acc/float(len(images))) + "Test-ACC:" + str(test_acc/float(len(test_images))))


I:0Train-ACC:0.127Test-ACC:0.2069
I:10Train-ACC:0.618Test-ACC:0.5982
I:20Train-ACC:0.632Test-ACC:0.6226
I:30Train-ACC:0.658Test-ACC:0.6447
I:40Train-ACC:0.681Test-ACC:0.6774
I:50Train-ACC:0.731Test-ACC:0.7064
I:60Train-ACC:0.753Test-ACC:0.7387
I:70Train-ACC:0.781Test-ACC:0.7622
I:80Train-ACC:0.829Test-ACC:0.7806
I:90Train-ACC:0.844Test-ACC:0.7992
I:100Train-ACC:0.84Test-ACC:0.8109
I:110Train-ACC:0.855Test-ACC:0.8216
I:120Train-ACC:0.868Test-ACC:0.8271
I:130Train-ACC:0.878Test-ACC:0.832
I:140Train-ACC:0.889Test-ACC:0.8367
I:150Train-ACC:0.887Test-ACC:0.8413
I:160Train-ACC:0.887Test-ACC:0.8435
I:170Train-ACC:0.898Test-ACC:0.8466
I:180Train-ACC:0.898Test-ACC:0.8478
I:190Train-ACC:0.909Test-ACC:0.849
I:200Train-ACC:0.913Test-ACC:0.8521
I:210Train-ACC:0.911Test-ACC:0.8518
I:220Train-ACC:0.902Test-ACC:0.8537
I:230Train-ACC:0.905Test-ACC:0.8556
I:240Train-ACC:0.91Test-ACC:0.857
I:250Train-ACC:0.913Test-ACC:0.8553
I:260Train-ACC:0.918Test-ACC:0.8585
I:270Train-ACC:0.908Test-ACC:0.8569
I:280Tr